<a href="https://colab.research.google.com/github/AndresMontesDeOca/Zubale/blob/main/Mid_LevelDataEngineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Challenge 1: CSV File Manipulation

In [17]:
# Load files

import pandas as pd

# Assuming 'orders.csv' is in the current working directory
data_orders = pd.read_csv('orders.csv', index_col='id', parse_dates=['created_date'])
data_products = pd.read_csv('products.csv', index_col='id')

### EDA

In [37]:
display(data_products.head())

# Not repeated products
print('Product name unique?:', data_products['name'].is_unique, '\n')

# Categories
print(data_products['category'].value_counts())

,name,category,price
id,,,
1,Product_1,Pants,92.55
2,Product_2,Shirts,43.11
3,Product_3,Jackets,59.02
4,Product_4,Shoes,49.65
5,Product_5,Pants,44.59


Product name unique?: True 

category
Shirts     6
Pants      4
Jackets    4
Shoes      3
Dresses    3
Name: count, dtype: int64
